In [1]:
!nvidia-smi

Mon Nov 16 02:29:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 41%   34C    P0    69W / 280W |     11MiB / 24220MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:05:00.0 Off |                  N/A |
| 41%   34C    P8    22W / 280W |     11MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
######### IMPORTING NECESSARY MODULES #########
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('/home/ironman/abhishek/Fixtures_Data/architectures/ResNeSt/')
sys.path.append('/home/ironman/abhishek/Fixtures_Data/Helper/')

from dataloader import mydataset, create_prime_dict 
from trainer import train, test_classify, eval_classify
from Load_model import load

##ResneSt
from resnest import resnest50 

In [3]:
device = torch.device('cuda:4' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:4


**Dataloading Scheme**

In [4]:
trainlist = 'Fixtures_Data_manual_clean_new_training_list_toys.txt'
validlist ='Fixtures_Data_manual_clean_new_validation_list_toys.txt'

In [5]:
prime_dict = create_prime_dict(trainlist)

Number of classes =  3323


In [6]:
#### Train Dataloader #### 
train_dataset = mydataset(trainlist, prime_dict, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 256, num_workers=16,pin_memory=True)


#### Validation Dataloader #### 
validation_dataset = mydataset(validlist, prime_dict, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 128, num_workers=16,pin_memory=True)

**Model Definition**

In [7]:
model = resnest50(pretrained=False, num_classes = 3323)

model = nn.DataParallel(model,device_ids=[4,5]).to(device)

In [8]:
# # Transfer Learning

# for param in model.module.parameters():
#     param.requires_grad = False

# fc_inputs = model.module.fc.in_features

# model.module.fc = nn.Sequential(
#     nn.Linear(fc_inputs, 4096),
#     nn.BatchNorm1d(4096),
#     nn.ReLU(),
#     nn.Linear(4096, 3323)
    
# )

# model.to(device)

**Hyperparameters**

In [19]:
## Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=1e-4, momentum=0.9)


# Epochs
num_Epochs = 120


# One Cycle Learning Rate
max_lr = 0.2
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, total_steps=None, epochs=num_Epochs, steps_per_epoch=len(train_dataloader), pct_start=0.3,
                anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=25.0)


In [12]:
modelname = 'Manually_cleaned_ResneSt_Backbone'
modelpath = '/home/ironman/abhishek/saved_model_checkpoints/Fixture_Data/' + modelname

In [ ]:
writer = SummaryWriter(modelname)

train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

loss 0.20492271423339845
loss 0.43938208803534506
loss 0.9347452097386122
loss 1.1989679657667875
loss 0.2776091805100441
loss 0.5510451436042786
loss 0.8386418461799622
loss 1.1200712467730045
loss 1.4100524219870567
loss 1.7099708062410355
loss 2.018659518957138
loss 2.317929205745459
loss 2.6368796426057814
loss 2.959420800507069
loss 3.2800405457615853
loss 3.6142437000572682
Epoch:  3
training loss =  0.30264150842106174
Validation Loss: 0.6799	Top 1 Validation Accuracy: 0.8566	 Top 5 Validation Accuracy: 0.9570
loss 0.28666002631187437
loss 0.5806642536818981
loss 0.8853486244380474
loss 1.202048552632332
loss 1.5311413823068143
loss 1.8649524691700936
loss 2.2053070992231367
loss 2.5503483997285366
loss 2.914724092781544
loss 3.273895172327757
loss 3.63197351038456
loss 3.997895499318838
Epoch:  4
training loss =  0.33590323442672665
Validation Loss: 0.6873	Top 1 Validation Accuracy: 0.8512	 Top 5 Validation Accuracy: 0.9564
loss 0.3391274267435074
loss 0.6796282076835632
loss 1

loss 3.7318756142258644
loss 4.369028292894363
loss 5.026759615242481
loss 5.691454794406891
loss 6.360365101993084
loss 7.027868030369282
loss 7.685860988795757
Epoch:  22
training loss =  0.6433297222212404
Validation Loss: 0.7825	Top 1 Validation Accuracy: 0.8284	 Top 5 Validation Accuracy: 0.9450
loss 0.595452312529087
loss 1.8227062743902207
loss 2.4496031990647316
loss 3.0880959993600845
loss 3.7371447917819025
loss 4.392096326947212
loss 5.052012121677398
loss 5.709081563055515
loss 6.355317443907261
loss 7.020349910855293
loss 7.679286493957043
Epoch:  23
training loss =  0.6426792451744019
Validation Loss: 0.7730	Top 1 Validation Accuracy: 0.8301	 Top 5 Validation Accuracy: 0.9470
loss 0.5958415678143502
loss 1.207460947930813
loss 1.818413731753826
loss 2.45471244096756
loss 3.07843397885561
loss 3.7310841196775435
loss 4.3741827484965325
loss 5.026402634680271
loss 5.672746029794216
loss 6.34478480219841
loss 7.025153829157352
loss 7.71072391718626
Epoch:  24
training loss =

In [16]:
#### Load saved model from checkpoint  #####
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)


In [17]:
#### Plot Loss Curves ####
plot_loss(epoch, train_loss, v_loss, title = modelname)
plot_acc(epoch, v_acc)

NameError: name 'plot_loss' is not defined

In [18]:
v_loss, top1_acc, accuracy_dict= eval_classify(model, validation_dataloader, criterion, device)
print('Validation Loss: {:.4f}\tTop 1 Validation Accuracy: {:.4f}\nAccuracy:{}\t'.format(v_loss, top1_acc, accuracy_dict))

Validation Loss: 0.6374	Top 1 Validation Accuracy: 0.8682
Accuracy:defaultdict(<class 'int'>, {'Top 1 Accuracy': 86.82492755320082, 'Top 5 Accuracy': 96.21813072622429, 'Top 10 Accuracy': 97.34508093551503, 'Top 20 Accuracy': 98.07979392910458, 'Top 30 Accuracy': 98.40177970318766, 'Top 50 Accuracy': 98.73547405087375, 'Top 100 Accuracy': 99.08965840236513})	
